In [ ]:
import glob
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_crossentropy
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
import torchvision
import torchvision.transforms as transforms
from keras.preprocessing.image import load_img
from keras.preprocessing.image import save_img
from keras.preprocessing.image import img_to_array, array_to_img

# Lung and Bronchus Cancer Classification

## Import the Data

In [ ]:
def train_directory():
    pwd = os.getcwd()
    
    os.mkdir(pwd+'/training')
    os.mkdir(pwd+'/training/LUAD_TRAIN')
    os.mkdir(pwd+'/training/LUSC_TRAIN')
    os.mkdir(pwd+'/training/MESO_TRAIN')

    
    train_dir = '/kaggle/input/histopathology-dataset/train/'
    cancers = ["LUAD","LUSC","MESO"]
    luad_train_dir = '/kaggle/working/training/LUAD_TRAIN/'
    lusc_train_dir = '/kaggle/working/training/LUSC_TRAIN/'
    meso_train_dir = '/kaggle/working/training/MESO_TRAIN/'
    cat_train_dir = [luad_train_dir,lusc_train_dir,meso_train_dir]

    for i in range(3):
        for f in glob.iglob(train_dir+cancers[i]+"/*"):
            for subf in glob.iglob(f+'/*'):
                shutil.copy(subf,cat_train_dir[i])
    
    labels = ["LUAD","LUSC","MESO"]
    dir_label_df = pd.DataFrame(columns = ["directory","label"])
    for i in range(3):
        filepaths_i = glob.glob(cat_train_dir[i]+"/*")
        series_i = pd.Series(filepaths_i)
        df_i = pd.DataFrame(series_i,columns = ["directory"])
        df_i["label"] = labels[i]
        dir_label_df = pd.concat([dir_label_df,df_i],axis=0)
    
    return dir_label_df.reset_index(drop=True)

In [ ]:
dir_label_df = train_directory()

In [ ]:
dir_label_df

In [ ]:
print(dir_label_df['label'].value_counts())

In [ ]:
dir_label_df.label.value_counts(normalize=True)

We can see that with our training data we have very imbalanced dataset with more data for the LUSC type of cancer. Hence, this will require some preprocessing.

## Data Preprocessing and Augmentation
We need to add more data to the imbalanced datasets. This means to the LUAD and MESO datasets. We augment our data using the following transformations:
* Random Rotation of 10 degrees
* Random Horizontal Flip
* Random Vertical Flip

This is randomly applied to the LUSC and for the LUAD and MESO data, it is applied to also create more data for these two classes so that it is more balanced.

In [ ]:
transform_train = transforms.Compose(
    [transforms.Resize((64,64)),
     transforms.RandomApply([
        #torchvision.centercrop(10),
        torchvision.transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip()],0.5),
     transforms.ToTensor()])

In [ ]:
def apply_transformation(train):
    
    aug_label_df = pd.DataFrame(columns = ["directory","label"])
    
    for i in range(len(train)):
        row = train.iloc[i]
        row_directory = row['directory']
        row_label = row['label']
        
        img = load_img(row_directory)
        trans_img = transform_train(img)
        new_dir = row_directory[:-4] + '_' + str(i) + '.jpg'
        save_img(new_dir,img_to_array(trans_img),data_format='channels_first')

        series_i = pd.Series(new_dir)
        df_i = pd.DataFrame(series_i,columns = ["directory"])
        df_i["label"] = "LUSC"
        aug_label_df = pd.concat([aug_label_df,df_i],axis=0)
                
        
    return aug_label_df

In [ ]:
def data_augmentation(train):
    
    aug_label_df = pd.DataFrame(columns = ["directory","label"])
    
    for i in range(len(train)):
        row = train.iloc[i]
        row_directory = row['directory']
        row_label = row['label']
        
        if row_label == "LUAD":
            
            for j in range(3):
                img = load_img(row_directory)
                trans_img = transform_train(img)
                new_dir = row_directory[:-4] + '_' + str(j) + '.jpg'
                save_img(new_dir,img_to_array(trans_img),data_format='channels_first')

                series_i = pd.Series(new_dir)
                df_i = pd.DataFrame(series_i, columns = ["directory"])
                df_i["label"] = "LUAD"
                aug_label_df = pd.concat([aug_label_df,df_i],axis=0)
                
        if row_label == "MESO":
            
            for z in range(26):
                
                img = load_img(row_directory)
                trans_img = transform_train(img)
                new_dir = row_directory[:-4] + '_' + str(z) + '.jpg'
                save_img(new_dir,img_to_array(trans_img),data_format='channels_first')

                series_i = pd.Series(new_dir)
                df_i = pd.DataFrame(series_i,columns = ["directory"])
                df_i["label"] = "MESO"
                aug_label_df = pd.concat([aug_label_df,df_i],axis=0)
                
        else:
            img = load_img(row_directory)
            trans_img = transform_train(img)
            new_dir = row_directory[:-4] + '_' + str(i) + '.jpg'
            save_img(new_dir,img_to_array(trans_img),data_format='channels_first')

            series_i = pd.Series(new_dir)
            df_i = pd.DataFrame(series_i,columns = ["directory"])
            df_i["label"] = "LUSC"
            aug_label_df = pd.concat([aug_label_df,df_i],axis=0)
                
        
    return aug_label_df

In [ ]:
data = data_augmentation(dir_label_df)

In [ ]:
data.label.value_counts()

In [ ]:
data.label.value_counts(normalize=True)

In [ ]:
! mkdir testing
%cd testing
! mkdir LUAD_TEST
! mkdir LUSC_TEST
! mkdir MESO_TEST

test_dir = '../kaggle/input/histopathology-dataset/dev/'
cancers = ["LUAD","LUSC","MESO"]
luad_test_dir = '../kaggle/working/testing/LUAD_TEST/'
lusc_test_dir = '../kaggle/working/testing/LUSC_TEST/'
meso_test_dir = '../kaggle/working/testing/MESO_TEST/'
cat_test_dir = [luad_test_dir,lusc_test_dir,meso_test_dir]

%cd /kaggle

for i in range(3):
    for f in glob.iglob(test_dir+cancers[i]+"/*"):
        for subf in glob.iglob(f+'/*'): 
            shutil.copy(subf,cat_test_dir[i])
            
            
labels = ["LUAD","LUSC","MESO"]
test_label_df = pd.DataFrame(columns = ["directory","label"])
for i in range(3):
    filepaths_i = glob.glob(cat_test_dir[i]+"/*")
    series_i = pd.Series(filepaths_i)
    df_i = pd.DataFrame(series_i,columns = ["directory"])
    df_i["label"] = labels[i]
    test_label_df = pd.concat([test_label_df,df_i],axis=0)
    
    
test_label_df = test_label_df.reset_index(drop=True)
test_label_df.head(),test_label_df.tail()

In [ ]:
plt.figure(figsize=(12,8))
labels = ["LUAD","LUSC","MESO"]
pos = 1
for i in range(3):
    for j in range(5):
        labels_i = labels[i]
        index_i = dir_label_df[dir_label_df["label"] == labels_i].index
        random = np.random.randint(index_i[0],index_i[-1])
        plt.subplot(3,5,pos)
        pos+=1
        plt.imshow(cv2.imread(dir_label_df.loc[random,"directory"]))
        plt.title(dir_label_df.loc[random, "label"], size = 15, color = "white") 
        plt.xticks([])
        plt.yticks([])

plt.show()

## Mosaics

Now, we want to get a mosaics for each of the classes and we'll keep the proportions similar to our transformed dataset.

In [ ]:
data.label.value_counts(normalize=True)

In [ ]:
luad_df = data[data.label == 'LUAD']
lusc_df = data[data.label == 'LUSC']
meso_df = data[data.label == 'MESO']

In [ ]:
def create_mosaic(df, ncol):
    n = df.shape[0]
    indices = np.random.choice(n, ncol**2)
    
    cols = []
    for i in range(ncol):
        col_i = []
        for j in range(ncol):
            ind = indices[0]
            img_j = img_to_array(load_img(df.directory.iloc[ind]))
            col_i.append(img_j)
            indices = indices[1:]  

        y_col = np.concatenate(col_i, axis=0)
        cols.append(y_col)

    y = np.concatenate(cols, axis=1)
    
    return y

In [ ]:
fp = '/kaggle/working/mosaic'
os.mkdir(fp)

In [ ]:
def get_mosaics(df, n, ncol, name):

    labels = [name] * n 
    d = []

    for i in range(n):
        new_mosaic = create_mosaic(df, ncol)
        new_fp = '/kaggle/working/mosaic/' + name + '_' + str(i) + '.jpg' 
        save_img(new_fp, img_to_array(new_mosaic))
        
        d.append(new_fp)
    
    mosaic_df = pd.DataFrame.from_dict({'directory': d, 'label': labels})
        
    return mosaic_df

In [ ]:
lusc = get_mosaics(lusc_df, 8000, 2, 'lusc')

In [ ]:
luad = get_mosaics(luad_df, 4000, 2, 'luad')

In [ ]:
meso = get_mosaics(meso_df, 4000, 2, 'meso')

In [ ]:
X_df = pd.concat([luad, lusc, meso], axis=0)
X_df.head()

In [ ]:
X_df.shape

In [ ]:
train_df, test_df = train_test_split(X_df, test_size = 0.2, random_state=42, shuffle=True)

In [ ]:
train_datagen = ImageDataGenerator()


train_gen = train_datagen.flow_from_dataframe(dataframe = train_df,
                                              x_col = 'directory', y_col ='label',
                                              target_size = (128,128), batch_size = 32, 
                                              class_mode = 'categorical', shuffle = True)


In [ ]:
test_datagen = ImageDataGenerator()


test_gen = test_datagen.flow_from_dataframe(dataframe = test_df,
                                              x_col = 'directory', y_col ='label',
                                              target_size = (128,128), batch_size = 32, 
                                              class_mode = 'categorical', shuffle = True)


In [ ]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
base_model_densenet = keras.applications.DenseNet121(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)
# Freeze the base_model
base_model_densenet.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))

x = base_model_densenet(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

x = keras.layers.Dense(256)(x)
x = keras.layers.Dense(128)(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32)(x)
outputs = keras.layers.Dense(3, activation="sigmoid")(x)
model_densenet = keras.Model(inputs, outputs)

model_densenet.summary()

callbacks = [EarlyStopping(monitor='val_loss',patience=4)]

model_densenet.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

model_densenet.fit(train_gen, validation_data = test_gen,
                    use_multiprocessing=True,
                    workers=6,
                    epochs = 15,
                    callbacks = callbacks,verbose=1)

model = Sequential()
model.add(Conv2D(32,(3, 3), activation = 'relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3), activation='relu'))

model.add(Conv2D(256,(2,2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(64, activation='relu'))

model.add(Flatten())
model.add(Dense(16, activation='relu'))

model.add(Dense(3, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])

model.fit(train_gen, epochs=12, validation_data = test_gen)

In [ ]:
model_densenet.save('densenet.h5')

In [ ]:
predictions = model_densenet.predict(test_gen)

In [ ]:
predictions.shape

In [ ]:
predicted_classes = np.argmax(predictions, axis=1)

In [ ]:
predicted_classes

In [ ]:
np.unique(predicted_classes)

In [ ]:
true_classes = test_gen.classes

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
np.sum(true_classes == predicted_classes)/len(predicted_classes)

In [ ]:
report = classification_report(true_classes, predicted_classes)
print(report)

In [ ]:
conf_mat = confusion_matrix(true_classes, predicted_classes)
print(conf_mat)

In [ ]:
val_data = apply_transformation(test_label_df)

In [ ]:
val_data.shape

In [ ]:
val_data

In [ ]:
y_val = np.asarray(test_label_df.label)

In [ ]:
validation_datagen = ImageDataGenerator()

validation_gen = validation_datagen.flow_from_dataframe(val_data,
                                            target_size = (128,128), x_col = 'directory', y_col ='label',
                                             class_mode = 'categorical',
                                            batch_size = 16, shuffle = False)

In [ ]:
validation_predictions = model.predict(validation_gen)
validation_predicted_classes = np.argmax(validation_predictions, axis=1)

validation_true_classes = validation_gen.classes
validation_class_labels = list(validation_gen.class_indices.keys()) 

In [ ]:
pd.DataFrame(validation_predicted_classes)[0].value_counts()

In [ ]:
validation_report = classification_report(validation_true_classes,
                               validation_predicted_classes, target_names=validation_class_labels)
print(validation_report)

In [ ]:
v_conf_mat = confusion_matrix(validation_true_classes, validation_predicted_classes)
print(v_conf_mat)